# Bellman-Ford Algorithm

Given a directed graph $G=(V,E)$ with edges lengths $c_e$ and a source vertex $s$

Compute the length of the shortest $s-v$ path for every other vertex in the graph.

## On Negative Cycles

When a cycle of edges in a graph has a negative total sum, the shortest path can be indefinately decremented by going one more loop around it. 

This invalidates the goal of the algorithm.

On solution is to compute the shortest cycle free path i.e; exclude any cycles. This however, is an NP-hard problem and there are no polynomial algorihtms.

Moving foward, the bellman-ford algorithm will assume that there are no negative cycles. Furthermore, the bellman-ford algorithm is able to quickly check if there is a negative cycle.

## Optimal Substructure

The key idea is to artificially restrict the number of edges in a path. Such that smaller subproblems relate to paths with fewer edges.

Let $G = (V, E)$ be a directed graph with edge lengths $c_e$ and source vertex $s$.

For every $v \in V$ and $i \in \{1, 2, 3, \cdots \}$, define
$$
p_{i,v} = \text{shortest} \; s \rightarrow v \; \text{path with at most} \; i \; \text{edges}
$$

Case 1:

if $p_{i,v}$ has less than $i-1$ edges, then it is the shortest $s \rightarrow v$ path.

Case 2:

if $p_{i,v}$ has $i$ edges. Consider the last hop $w \rightarrow v$. Then $p_{i-1, w}$ is a shortest path.

With these two cases we can develop a recurrence. Let $L_{i, v}$ be the minminum length of a $s \rightarrow v$ path with $\leq i$ edges

For every $v \in V$, $i \in \{1, 2, 3, \cdots\}$
$$
L_{i,v} = \min
\begin{cases}
L_{(i-1), v} \\
\min\limits_{(w, v) \in E}{(L_{(i-1, w)} + c_{(w,v)})}
\end{cases}
$$

Suppose an input graph does not have any negative cycles. Then the shortst paths will not have any cycles, and the longest shortest path will be $\leq (n-1)$ edges. Therefore, we only need to solve subproblems up to $i = n-1$

Therefore the subproblem space is
$$
i \in \{1, 2, \cdots, n-1\} \\
v \in V
$$

## Psuedocode

```
Let A = 2D array

A[0, s] = 0
A[0, v] = +inf for all v != s

for i= 1 to n-1:
    for each v in V:

        A[i, v] = min(
            A[i-1, v]

            min over edges (w, v)(
                A[i-1, w] + c_wv
            )
        )
```

Running time is $O(mn)$. Note that for sparse graphs $m = O(n)$ and for dense graphs $m = O(n^2)$. This aligns with our intution that the running time should be somewhere between $O(n^2)$ and $O(n^3)$

## Checking for Negative Cycles

We claim that after running the Bellman-Ford algorithm for 1 more $n^{th}$ iteration,
$$
G \; \text{has no negative cycles} \iff A[n-1, v] = A[n, v] \quad \forall v \in V
$$

Proof:

The forward direction follows from the proof of correctness of the Bellman-Ford algorithm. 

For the reverse direction, assume
$$
A[n-1, v] = A[n, v] \quad \forall v \in V
$$

Let $d(v)$ denote the common value of $A[n-1, v]$ and $A[n, v]$. This indicates that
$$
d(v) \leq d(w) + c_{w,v} \quad \forall (w, v) \in E
$$

Consider an arbitrary cycle $C$
$$
\sum_{(w,v) \in C}{c_{w, v}} \geq \sum_{(w,v) \in C}{d(v) - d(w)} = 0
$$

Therefore the cycles must have positive overall sum.

## Optimisations

1. Stop early

If at some iteration $j < n-1$
$$
A[j, v] = A[j-1, v] \quad \forall v \in V
$$
Then we can stop the algorithm early as it will only continue to further inherit previous values till completion

2. Space Optimisation

We only need to store the previous array values for the next array. However, in order to maintain information regarding the paths themselves, we will need to store at each vertex a predecessor pointer. Let
$$
B[i, v] = 2^{nd} \; \text{to last vertex on the shortest} \; s \rightarrow v \text{path}
$$
Then, we need only to hop through $B$ values to reconstruct the shortest paths.

To reconstruct a negative cost cycles, we can use depth-first search to check for a cycle of predecessor pointers after each round.


## Creating a routing protocol

1. Switch from source-driven to destination-driven

every vertex $v$ stores shortest-path distances from $v$ to any relevant destination $t$ and the first hop of a shortest path.

2. Hanndling Asynchrony

we cannot assume all $A[i-1, v]$ get computed before all $A[i, v]$. We can switch from a "pull-based" to a "push-based" approach. 